In [1]:
import mne 
import pickle
import argparse
import numpy as np
import matplotlib.pyplot as plt
import os

from mne_connectivity import spectral_connectivity_epochs

from src.params import SUBJ_CLEAN, RESULT_PATH, ACTIVE_RUN, PASSIVE_RUN, FIG_PATH, PREPROC_PATH, FREQS_LIST, FREQS_NAMES
from src.utils import get_bids_file
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.model_selection import cross_val_score, ShuffleSplit, RandomizedSearchCV, GridSearchCV, LeaveOneGroupOut
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn import svm


In [ ]:
all_epochs = []
subj_list = SUBJ_CLEAN
task='LaughterActive'
cond1 = 'LaughReal'
freq_name = 'alpha'
stage = 'psd_epo'


if task == 'LaughterActive' :
    run_list = ACTIVE_RUN
elif task == 'LaughterPassive':
    run_list = PASSIVE_RUN

for i, freq in enumerate(FREQS_NAMES) : 
    if freq == freq_name :
        fmin, fmax = FREQS_LIST[i][0], FREQS_LIST[i][1]


for subj in subj_list : 
    for run in run_list : 
        _, psd_path = get_bids_file(RESULT_PATH, stage=stage, subj=subj, task=task, run=run, measure=freq_name)
        epochs_run = mne.read_epochs(psd_path)
        sfreq = epochs_run.info['sfreq'] 
        all_epochs.append(epochs_run[cond1])

epochs = mne.concatenate_epochs(all_epochs)

In [ ]:
# Compute connectivity
method = 'wpli'

con_wpli = spectral_connectivity_epochs(epochs, method=method, mode='multitaper', sfreq=sfreq, fmin=fmin,
                                        fmax=fmax, faverage=True, tmin=tmin, mt_adaptive=False, n_jobs=1)

In [ ]:
# Plot connectivity matrice
fig, axs = plt.subplots(1, figsize=(14, 5), sharey=True)

axs[1].imshow(con_wpli.get_data('dense'), vmin=0, vmax=1)
axs[1].set_title("wPLI")
axs[1].set_xlabel("Sensor 2")